In [98]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import random as rnd

In [99]:
column_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']
df=pd.read_csv("C:\\Users\\kiril\\Downloads\\housing (1).csv",header=None,delimiter=r"\s+",names=column_names)
X=df.drop(['MEDV'],axis=1)
y=df["MEDV"]

In [100]:
reg=LinearRegression().fit(X,y)
print(f"R^2{reg.score(X,y)}")
print(f'Коэффиценты : {reg.coef_}')
y_pred = reg.predict(X)
mse = mean_squared_error(y, y_pred)
print(f"MSE: {mse:.4f}")

R^20.7406426641094095
Коэффиценты : [-1.08011358e-01  4.64204584e-02  2.05586264e-02  2.68673382e+00
 -1.77666112e+01  3.80986521e+00  6.92224640e-04 -1.47556685e+00
  3.06049479e-01 -1.23345939e-02 -9.52747232e-01  9.31168327e-03
 -5.24758378e-01]
MSE: 21.8948


**Ручная реализация**

In [101]:
# Нормализация признаков 
X_mean = X.mean() # среднее
X_std = X.std() # стандартное отклонение
X_norm = (X - X_mean) / X_std # стандартизация

X_norm['w_0'] = 1

X_values = X_norm.values
y_values = y.values

weights = [rnd.uniform(-0.01, 0.01) for _ in range(X_norm.shape[1])]

def predict(X_vals, weights):
    predictions = []
    for row in X_vals:
        pred = 0.0
        for j in range(len(weights)):
            pred += weights[j] * row[j]
        predictions.append(pred)
    return predictions

def MSE(y_vals, predictions):
    n = len(predictions)
    mse = 0.0
    for i in range(n):
        mse += (y_vals[i] - predictions[i]) ** 2
    return mse / n

def r2_score(y_vals, predictions):
    n = len(y_vals)
    y_mean = sum(y_vals) / n
    sse = sum((y_vals[i] - predictions[i]) ** 2 for i in range(n))
    sst = sum((y_vals[i] - y_mean) ** 2 for i in range(n))
    return 1 - (sse / sst) if sst != 0 else 0.0

def calculate_gradient(X_vals, y_vals, predictions, weights):
    n_samples = len(y_vals)
    n_features = len(weights)
    grad = [0.0] * n_features
    for j in range(n_features):
        for i in range(n_samples):
            grad[j] += -2 * (y_vals[i] - predictions[i]) * X_vals[i][j]
        grad[j] /= n_samples
    return grad

learning_rate = 0.1 
num_iterations = 10000
tolerance = 1e-4

prev_mse = 10.0**10
for itter in range(num_iterations + 1):
    predictions = predict(X_values, weights)
    error = MSE(y_values, predictions)

    # остановка, если сошлись
    if abs(prev_mse - error) < tolerance and itter > 0:
        print(f"\n Сошлись на итерации {itter}, MSE = {error}")
        break
    prev_mse = error

    if itter < 11:
        score = r2_score(y_values, predictions)
        print(f"Iteration {itter}, MSE = {error}, R² = {score}")
    elif itter >= 11 and itter % 10 == 0:
        score = r2_score(y_values, predictions)
        print(f"Iteration {itter}, MSE = {error}, R² = {score}")

    grad = calculate_gradient(X_values, y_values, predictions, weights)
    for j in range(len(weights)):
        weights[j] -= learning_rate * grad[j]

final_predictions = predict(X_values, weights)
final_mse = MSE(y_values, final_predictions)
final_r2 = r2_score(y_values, final_predictions)
print(f"\nФинальные метрики: MSE = {final_mse}, R^2 = {final_r2}")
print(f'Финальные веса (коэффиценты) : {weights}')

Iteration 0, MSE = 591.5197722335695, R² = -6.006904551112921
Iteration 1, MSE = 367.1975673737244, R² = -3.3496742235229826
Iteration 2, MSE = 242.0025766569986, R² = -1.8666648780920405
Iteration 3, MSE = 163.1293176868576, R² = -0.9323640766967412
Iteration 4, MSE = 112.86491171610547, R² = -0.3369522046209237
Iteration 5, MSE = 80.76283372538887, R² = 0.04331605847361064
Iteration 6, MSE = 60.22903579054212, R² = 0.2865511436813769
Iteration 7, MSE = 47.072739972186916, R² = 0.44239531554621947
Iteration 8, MSE = 38.62619634694241, R² = 0.5424496632570682
Iteration 9, MSE = 33.18933908477666, R² = 0.6068524806813655
Iteration 10, MSE = 29.677764184987286, R² = 0.6484491800680977
Iteration 20, MSE = 22.94505624712305, R² = 0.7282021217373191
Iteration 30, MSE = 22.4943454665372, R² = 0.733541059788025
Iteration 40, MSE = 22.28381407127995, R² = 0.7360349297494803
Iteration 50, MSE = 22.160164240622013, R² = 0.7374996357523074
Iteration 60, MSE = 22.082836327341305, R² = 0.7384156310